In [104]:
import googlemaps
from googlemaps import Client as GoogleMaps
import gmplot
from gmplot import *
import gmaps
from datetime import datetime 
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
import pandas as pd
import os
os.curdir
import local_config
import lydia_functions
import npi_rename
LYDIA_GOOGLE_API_KEY = local_config.lydia_google_api_key

In [ ]:
'''
tax_df=pd.read_pickle('data/taxonomy.pkl')
tax=tax_df.drop_duplicates(subset=None, keep='first', inplace=False).reset_index(drop=True)
tax=tax.rename(columns={'provider_taxonomy_code':'taxonomy_code','provider_taxonomy_desc':'specialty'})
tax=tax[['taxonomy_code', 'specialty']].reset_index(drop=True)
tax['taxonomy_code']=tax.taxonomy_code.astype(str)
tax=tax[tax.taxonomy_code!='nan']
tax['specialty']=tax['specialty'].str.split('/')
def name_cleaner(raw):
    col=[]
    for x in raw.specialty:
        if len(x)==1:
            col.append(x)
        else:
            col.append(x[1])
    return col
def med(spec):
    col=[]
    spec=tax.specialty
    for x in spec:
        if 'Internal Medicine,'in x:
            col.append(x.split(',')[1])
        elif ',' in x:
            col.append(x.split(',')[0])
        else:
            col.append(x)
    return col
tax['specialty']=name_cleaner(tax)
tax['specialty']=med(tax.specialty)
tax=tax.rename(columns={'taxonomy_code':'taxonomy'})
tax.to_pickle('data/tax.pkl')
'''

In [ ]:
'''
npi_file_path='data/npidata_pfile_20050523-20200209.csv'
npi = pd.read_csv(npi_file_path, usecols=lydia_functions.npi_finder(), 
                  dtype={'Provider Business Practice Location Address Postal Code': 'str'})
npi=npi.rename(columns=lydia_functions.npi_rename_cols())
for i in range(1,16):
    npi['tax{}'.format(i)]=npi['tx{}'.format(i)]+npi['sw{}'.format(i)]
    npi= npi.drop(['tx{}'.format(i),'sw{}'.format(i)], axis=1)
    npi.loc[npi['tax{}'.format(i)].str[-1:]=='Y','taxonomy']=npi['tax{}'.format(i)].str[:-1]
    npi= npi.drop(["tax{}".format(i)],axis=1)
npi=npi.drop_duplicates(subset=None, keep='first', inplace=False).reset_index(drop=True)
npi['zipcode']=npi.zipcode.str[:5]
npi['address']=npi['street_address']+', ' + npi['city'] + ', '+ npi['state']+', '+npi['zipcode']
cols = npi[['firstname', 'lastname', 'name_suffix', 'credential']]
npi['name']=npi['name_prefix'].str.cat(cols, sep = " ", na_rep = '')
business=npi[['business_name']]
npi['name']=npi['name'].str.cat(business, sep = " ", na_rep = '')
npi= npi.drop(columns=['business_name','lastname', 'firstname', 'name_prefix',
       'name_suffix', 'credential', 'street_address','city', 'state'])
npi=npi[['name',  'address','entity_type', 'taxonomy','npi','other_business_name','zipcode']].reset_index(drop=True)
npi.to_pickle('data/npi.pkl')
'''

# data imports

In [2]:
'''
#use this taxonomy and npi datasets 
tax_df=pd.read_pickle('data/tax.pkl')
npi=pd.read_pickle('data/npi.pkl')
hop_df = pd.read_pickle('data/hop.pkl')
#npi=npi.merge(tax_df, on ='taxonomy',how="left")

# Mapping

In [118]:
'''
ADDR_MAP=pd.read_pickle('data/ADDR_MAP.pkl')
tn_add=pd.DataFrame.from_dict(ADDR_MAP, orient='index')
midlat=tn_add.lat.mean()
midlong=tn_add.lng.mean()
lat= tn_add.lat[:10]
lng=tn_add.lng [:10]
gmap_scatter=gmplot.GoogleMapPlotter(midlat, midlong, 11, apikey = LYDIA_GOOGLE_API_KEY)
gmap_scatter.coloricon = "http://www.googlemapsmarkers.com/v1/%s/"
for i, row in tn_add.iterrows():
    gmap_scatter.marker(row['lat'], row['lng'])
gmap_scatter.plot(lat, lng, 'cornflowerblue', edge_width = 3.0)
gmap_scatter.draw('providers.html')

# from_npi

In [8]:
'''
hop['npi']=hop['from_npi']
hop=hop.merge(npi, on ='npi',how="inner")
hop=hop.rename(columns={'name':'from_name',
                        'address':'from_address',
                        'entity_type':'from_entity_type', 
                        'taxonomy':'from_taxonomy', 
                        'other_business_name':'from_other_business_name',
                        'zipcode':'from_zipcode',
                        'specialty':'from_specialty'})
#hop.to_pickle('data/from_hop.pkl')
'''

From entity-1 and over 50 transaction, less than 50 avg day wait

In [43]:
'''
#only referalls from doctors
doctors=npi[npi['entity_type']==1].reset_index(drop=True)
hop=hop[hop.from_npi.isin(doctors.npi)]
#Filter transaction_count > 50, average_day_wait < 50
hop=hop[(hop['transaction_count'] > 50) & (hop1['average_day_wait'] < 50)].reset_index(drop=True)
hop=hop.drop(columns=['average_day_wait','std_day_wait','from_taxonomy','npi'])

# to_npi

In [55]:
'''
hop1['npi']=hop1['to_npi']
hop1=hop1.merge(npi, on ='npi',how="inner")
hop1=hop1.drop(columns='npi')
hop1=hop1.rename(columns={'name':'to_name',
                        'address':'to_address',
                        'entity_type':'to_entity_type', 
                        'other_business_name':'to_other_business_name',
                        'zipcode':'to_zipcode',
                        'specialty':'to_specialty'})
hop1=hop1.drop(columns=['to_taxonomy','from_other_business_name','from_entity_type'])                        
#hop1.to_pickle('data/hopfinal.pkl')
'''

In [13]:
'''
cbsa=pd.read_excel('data/CBSA_ZIP_122019 .xlsx')
cbsa=cbsa[['CBSA','ZIP']]
cbsa=cbsa.rename(columns={'CBSA':'cbsa', 'ZIP':'zipcode'})
cbsa.zipcode=cbsa.zipcode.astype(str)
cbsa.to_csv('data/cbsa.csv')
'''

In [43]:
'''
hop['zipcode']=hop['to_zipcode']
hop=hop.merge(cbsa, on ='zipcode',how="inner")
hop=hop.rename(columns={'cbsa':'to_cbsa'})
hop['zipcode']=hop['from_zipcode']
hop=hop.merge(cbsa, on ='zipcode',how="inner")
hop=hop.rename(columns={'cbsa':'from_cbsa'}) '''

# export final clean data

In [ ]:
hop.to_csv('hop_to_from_full.csv')
hop.to_pickle('hop_to_from_full.pkl')

# payments recived

In [228]:
ownership=pd.read_csv('data/OP_DTL_OWNRSHP_PGYR2017_P01172020.csv')
research_df=pd.read_csv('data/OP_DTL_RSRCH_PGYR2017_P01172020.csv')
paydf2=ownership[['Physician_Profile_ID','Physician_First_Name','Physician_Last_Name','Physician_Name_Suffix',
                  'Recipient_Primary_Business_Street_Address_Line1','Recipient_City','Recipient_State',
                  'Recipient_Zip_Code','Physician_Primary_Type','Physician_Specialty', 'Record_ID',
                  'Total_Amount_Invested_USDollars', 'Value_of_Interest','Terms_of_Interest',
                  'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country',
         'Interest_Held_by_Physician_or_an_Immediate_Family_Member'         
                 ]]
paydf3=research_df[['Physician_Profile_ID','Physician_First_Name','Physician_Last_Name','Physician_Name_Suffix',
                   'Recipient_Primary_Business_Street_Address_Line1', 'Recipient_City',
       'Recipient_State', 'Recipient_Zip_Code', 'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State',
                    'Total_Amount_of_Payment_USDollars', 'Date_of_Payment',
       'Form_of_Payment_or_Transfer_of_Value',
                   ]]
paydf2.to_pickle('data/ownership_payment.pkl')
paydf3.to_pickle('data/research_payment.pkl')


pay3=pay3[pay3['Physician_Profile_ID'].notna()].reset_index(drop=True)
pay3['name']=pay3['Physician_First_Name'] + ' ' + pay3['Physician_Last_Name']
pay3['address']=pay3['Recipient_Primary_Business_Street_Address_Line1'] +', ' + pay3['Recipient_City']+', '+pay3['Recipient_State']
pay3=pay3.rename(columns={'Physician_Profile_ID':'id',
                          'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name':'payer_name',
                          'Total_Amount_of_Payment_USDollars':'amount',
                         'Recipient_Zip_Code':'zipcode'})
pay3=pay3[['id','name','address','payer_name','amount','zipcode',]]
pay3['name']=pay3.name.str.lower()
pay3['address']=pay3.address.str.lower()
pay3['zipcode']=pay3.zipcode.astype(str)
pay3['zipcode']=pay3.zipcode.str[:5]
tn=pay3[pay3.address.str.contains(', tn')].reset_index()
tn.to_pickle('data/tn_doc_pay,pkl')

In [62]:
ownership_pay=pd.read_pickle('data/ownership_payment.pkl')
research_pay=pd.read_pickle('data/research_payment.pkl')

In [102]:
def col_renamer(df):
    df=df.rename(columns={'Physician_Profile_ID':'id',
                          'Physician_First_Name':'first_name',
                          'Physician_Last_Name':'last_name',
                          'Recipient_Primary_Business_Street_Address_Line1':'street_address',
                          'Recipient_City':'city','Recipient_State':'state',
                          'Total_Amount_of_Payment_USDollars':'amount',
                         'Recipient_Zip_Code':'zipcode',
                          'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State':'payer_state',
                          'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name':'payer_name',
                          'Physician_Specialty':'specialty',
                          'Total_Amount_Invested_USDollars':'dollar_amount'
                         })
    df=df.drop(columns=['Physician_Name_Suffix','Record_ID','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
                        'Physician_Name_Suffix','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country',
                        'Interest_Held_by_Physician_or_an_Immediate_Family_Member','Physician_Primary_Type',
                        'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name', 'Date_of_Payment',
                        'Form_of_Payment_or_Transfer_of_Value', 'Expenditure_Category1',
       'Expenditure_Category2', 'Expenditure_Category3',
       'Expenditure_Category4', 'Expenditure_Category5'
                       ])
    return df
ownership_pay=col_renamer(ownership_pay)
ownership_pay['specialty']=ownership_pay.specialty.str.split('|').str[1]

In [119]:
def col2_renamer(df):
    df=df.rename(columns={'Physician_Profile_ID':'id',
                          'Physician_First_Name':'first_name',
                          'Physician_Last_Name':'last_name',
                          'Recipient_Primary_Business_Street_Address_Line1':'street_address',
                          'Recipient_City':'city','Recipient_State':'state',
                          'Total_Amount_of_Payment_USDollars':'amount',
                         'Recipient_Zip_Code':'zipcode',
                          'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State':'payer_state',
                          'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name':'payer_name',
                          'Physician_Specialty':'specialty',
                          'Total_Amount_Invested_USDollars':'dollar_amount'
                         })
    df=df.drop(columns=['Physician_Name_Suffix','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
                        'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name', 'Date_of_Payment',
                        'Form_of_Payment_or_Transfer_of_Value', 'Expenditure_Category1',
       'Expenditure_Category2', 'Expenditure_Category3',
       'Expenditure_Category4', 'Expenditure_Category5'
                       ])
    return df
research_pay = col2_renamer(research_pay)

# export clean financial relationship datasets

In [106]:
ownership_pay.to_pickle('data/ownership_pay.pkl')
ownership_pay.to_csv('ownership_pay.csv')
research_pay.to_pickle('data/research_pay.pkl')
research_pay.to_csv('research_pay.csv')